In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn import linear_model
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, cross_val_score
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import glob
import os
%load_ext autoreload
%autoreload 2

In [ ]:
cwd = os.getcwd()

In [ ]:
cwd = os.chdir('/content/drive/MyDrive/Kaggle/classify-cancer-type-clustering-only')
print(os.getcwd())

In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
train_df = pd.read_csv(cwd +'/train_dataset.csv', index_col=0)

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df = pd.read_csv(cwd + '/test_dataset.csv', index_col = 0)

In [ ]:
label_mapping = {k: num for num, k in enumerate(train_df["label"].unique())}
label_mapping
X = train_df.drop('label',axis =1)
y = train_df["label"].map(label_mapping)

# Подбир гиперпараметров для LGBMClassificator - датасет с признаками - предсказение

In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def objective(trial, X, y):
    param_grid = {}  # to be filled in later
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scores = []
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
        )
        preds = model.predict(X_test)
        scores.append(f1_score(preds, y_test))

    return np.mean(scores)

In [ ]:
import optuna
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def objective(trial, X, y):
    param_grid = {
        "objective": 'binary',
        "n_estimators": trial.suggest_int("n_estimators", 10, 1500, 90),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5),
        #"num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20), WRONG
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 10000, step=100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.001, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.001, 1),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        )
    }

    param_grid['num_leaves'] = 2**param_grid['max_depth']
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scores = []

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = LGBMClassifier(**param_grid)

        model.fit(
            X_train,
            y_train,
        )

        preds = model.predict(X_test)
        scores.append(f1_score(preds, y_test, average='weighted'))

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train_scaler, y)
study.optimize(func, n_trials=50)

In [ ]:
best_trial = study.best_trial

In [ ]:
best_trial.values

In [ ]:
best_trial.params

In [ ]:
best_params = best_trial.params

In [ ]:
modelLGBM = LGBMClassifier(**best_params)

In [ ]:
# Сохраняю, т.к. сбрасываются переменные при отключении Colab
#model = LGBMClassifier(
#    n_estimators = 1180,
#    learning_rate = 0.30074249832612265,
#    max_depth = 5,
#    min_data_in_leaf = 300,
#    lambda_l1 = 0.734811756051383,
#    lambda_l2 = 0.8013373959927099,
#    min_gain_to_split = 7.514296384060523,
#    bagging_fraction = 0.8
#)

In [ ]:
modelLGBM = LGBMClassifier(
    n_estimators = 1180,
    learning_rate = 0.30074249832612265,
    max_depth = 5,
    min_data_in_leaf = 300,
    lambda_l1 = 0.734811756051383,
    lambda_l2 = 0.8013373959927099,
    min_gain_to_split = 7.514296384060523,
    bagging_fraction = 0.8
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
modelLGBM.fit(X_train,y_train)

In [ ]:
feature_name = pd.DataFrame(data = modelLGBM.feature_name_, columns=['Name']).reset_index()
feature_importance = pd.DataFrame(data = modelLGBM.feature_importances_, columns=['Importance']).reset_index()

In [ ]:
feature = pd.merge(feature_name,feature_importance, on ='index' )
feature_rank = feature[feature['Importance']>0]

In [ ]:
feature_rank.head()

In [ ]:
feature_final = list(feature_rank['Name'])
len(feature_final)

In [ ]:
import joblib
joblib.dump(modelLGBM, 'modelLGBM.pkl')

In [ ]:
# modelLGBM_load = joblib.load('modelLGBM.pkl')

In [ ]:
X_train_feature = X[feature_final]
X_test_feature = test_df[feature_final]

При необходимости стандартизируем

In [ ]:
scaler = preprocessing.StandardScaler()
X_train_scaler_feature_LGBM = scaler.fit_transform(X_train_feature)
X_test_scaler_feature_LGBM = scaler.fit_transform(X_test_feature)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_feature, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scores = cross_val_score(modelLGBM, X_train, y_train, cv=5)

In [ ]:
scores

In [ ]:
scores.mean()

# Подбор гиперпараметров для XGBoost

In [ ]:
!pip install xgboost

In [ ]:
!pip install optuna

In [ ]:
import xgboost as xgb
import optuna  # pip install optuna
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')

In [ ]:
study.optimize(objective, n_trials=100)

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Best trial:
  Value: 0.9418181818181818
  Params: 
    max_depth: 5
    learning_rate: 0.3133913754259897
    n_estimators: 391
    min_child_weight: 6
    gamma: 2.1257856527281565e-07
    subsample: 0.37862385631000617
    colsample_bytree: 0.1578019328479794
    reg_alpha: 9.661666006983265e-06
    reg_lambda: 0.0010438083055327087

In [ ]:
params = trial.params

In [ ]:
# Сохранил лучшие параметры, добавив к ним мультикласс:softmax
# и количество классов = 8
modelXGBC = XGBClassifier(
    max_depth = 5,  learning_rate = 0.3133913754259897,
    n_estimators = 391, min_child_weight = 6, 
    gamma = 2.1257856527281565e-07, subsample = 0.37862385631000617,
    colsample_bytree = 0.1578019328479794,
    reg_alpha = 9.661666006983265e-06, reg_lambda = 0.,
    objective= "multi:softmax", num_class = 8
)

In [ ]:
modelXGBC

In [ ]:
#modelXGBC = XGBClassifier(**params)
modelXGBC = modelXGBC.fit(X_train, y_train) 

In [ ]:
y_pred = modelXGBC.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy after tuning: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import joblib
joblib.dump(modelXGBC, 'modelXGBCFullfeature.pkl')

In [ ]:
import joblib
modelXGBC = joblib.load('modelXGBCFullfeature.pkl')

Важные признаки по мнению XGBClassifier

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
from numpy import loadtxt
from numpy import sort

In [ ]:
feat_names = list(train_df.columns)
def get_xgb_imp(xgb, feat_names):
    imp_vals = modelXGBC.get_booster().get_score(importance_type='weight')
    feats_imp = pd.DataFrame(imp_vals,index=np.arange(2)).T
    feats_imp.iloc[:,0]= feats_imp.index
    feats_imp.columns=['feature','importance']
    feats_imp.sort_values('importance',inplace=True,ascending=False)
    feats_imp.reset_index(drop=True,inplace=True)
    return feats_imp
feat_names = np.array(train_df.columns)
feature_importance_df = get_xgb_imp(modelXGBC, feat_names)

In [ ]:
feature_importance_df

In [ ]:
feature_importance_df_select = feature_importance_df['feature']

In [ ]:
feature_importance_df_select

In [ ]:
feature_importance_df_select = pd.DataFrame(data = feature_importance_df_select)

In [ ]:
feature_final_XGBC = list(feature_importance_df_select['feature'])
feature_final_XGBC

In [ ]:
X_train_feature_XGBC = train_df[feature_final_XGBC]
X_train_feature_XGBC.head()

In [ ]:
X_test_feature_XGBC = test_df[feature_final_XGBC]
X_test_feature_XGBC.head()

Дальше была игра с исключением и включением признаков, которые могут потенциально улучшить предсказание brest1/brest3

In [ ]:
X_train_feature_XGBC = X_train_feature_XGBC.drop(columns='ROPN1')
X_test_feature_XGBC = X_test_feature_XGBC.drop(columns='ROPN1')

In [ ]:
#X_train_feature_XGBC['MUCL1'] = train_df['MUCL1']
#X_test_feature_XGBC['MUCL1'] = test_df['MUCL1']

In [ ]:
X_train_feature_XGBC['TRBV7-6'] = train_df['TRBV7-6']
X_test_feature_XGBC['TRBV7-6'] = test_df['TRBV7-6']

In [ ]:
#X_train_feature_XGBC['ALB'] = train_df['ALB']
#X_test_feature_XGBC['ALB'] = test_df['ALB']

In [ ]:
#X_train_feature_XGBC['SMR3B'] = train_df['SMR3B']

In [ ]:
#X_train_feature_XGBC['IGLV2-11'] = train_df['IGLV2-11']

In [ ]:
#X_train_feature_XGBC['CSN1S1'] = train_df['CSN1S1']

In [ ]:
#X_train_feature_XGBC['GAPDH'] = train_df['GAPDH']

In [ ]:
#X_train_feature_XGBC['IGKV1D-39'] = train_df['IGKV1D-39']

In [ ]:
#X_train_feature_XGBC['LALBA'] = train_df['LALBA']

In [ ]:
#X_train_feature_XGBC = X_train_feature_XGBC.drop(columns='VGLL1')

In [ ]:
#X_train_feature_XGBC['TRBV7-6'] = np.where(train_df['TRBV7-6'] > 2.1, 100 , 0)

In [ ]:
#X_train_feature_XGBC = X_train_feature_XGBC.drop(columns='PTCHD3')

In [ ]:
#X_train_feature_XGBC['PTCHD3'] = np.where(X_train_feature_XGBC['PTCHD3'] > 0.037, 10 , 0.0)

In [ ]:
#X_train_feature_XGBC['NXN'] = train_df['NXN']

In [ ]:
#X_train_feature_XGBC['NXN'] = np.where(train_df['NXN'] > 25, 100 , 0)

In [ ]:
#X_train_feature_XGBC['VGLL1'] = np.where(X_train_feature_XGBC['VGLL1'] > 29.027407, 150 , 0)

In [ ]:
# Пробуем с/без стандартизацией
scaler = preprocessing.StandardScaler()
X_train_scaler_feature_XGBC = scaler.fit_transform(X_train_feature_XGBC)
X_test_scaler_feature_XGBC = scaler.fit_transform(X_test_feature_XGBC)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_feature_XGBC, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
modelXGBC.fit(X_train, y_train)

In [ ]:
scores = cross_val_score(modelXGBC, X_train, y_train, cv=5)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
print(classification_report(y_test, y_pred))

Повторно вижу, что 0 и 5 класс не могут определится

# OneVsRest

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_predict

In [ ]:
ovr = OneVsRestClassifier(
    estimator= modelXGBC,
    n_jobs = 3
    )

#fit model to training data
ovr.fit(X_train, y_train)

#Predications
ovr_pred = ovr.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
   
# Evaluating the model
print(f"Test Set Accuracy : {accuracy_score(y_test, ovr_pred) * 100} %\n\n")
print(f"Classification Report : \n\n{classification_report(y_test, ovr_pred)}")

Наверное это глупый шаг, потому что в XGBC уже есть мультикласс

In [ ]:
ovr_predictions = ovr.predict(X_test_scaler)

# Сама модель

In [ ]:
model_pred = modelXGBC.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
   
# Evaluating the model
print(f"Test Set Accuracy : {accuracy_score(y_test, model_pred) * 100} %\n\n")
print(f"Classification Report : \n\n{classification_report(y_test, model_pred)}")

# Создаем Submit

In [ ]:
model_pred = modelXGBC.predict(X_test_feature_XGBC)

In [ ]:
predictions_df = pd.DataFrame(
    data=model_pred, index=test_df.index, columns=["Predicted"]
)
predictions_df
predictions_df.index.name = "Id"
predictions_df["Predicted"].map({v: k for k, v in label_mapping.items()})

predictions_df["Predicted"] = predictions_df["Predicted"].map(
    {v: k for k, v in label_mapping.items()})

In [ ]:
predictions_df.head()

In [ ]:
predictions_df.to_csv("submission_XGBoost_FeatureSelection.csv")

# Clustering чтобы понять, что не так с 0 и 5 классом

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=5, n_iter=50000).fit_transform(X)

In [ ]:
plt.figure(figsize = (10,10))
sns.scatterplot(x=X_embedded[:, 0], y=X_embedded[:, 1], hue=y, palette="deep")

Видим, что классы наслаиваются друг на друга. Поэтому попробуем поискать различия

#  Разбираемся почему 5 и 0 класс предсказываются плохо

Сразу стоит сказать, что это оказалось не очень информативным занятием, однако удалось выяснить, что некоторые гены экспрессируется в одинаковом количестве у этих двух типов, а некоторые преобладают в определенном типе (brest1\brest2). Но выбор этих фич для оценки - довольно трудоемкое и длительное дело, с которым ручным методом я не справился

In [ ]:
train_df_copy = train_df[feature_final_XGBC].copy()
label_mapping = {k: num for num, k in enumerate(train_df["label"].unique())}
label_mapping
y = train_df["label"].map(label_mapping)
train_df_copy['label_num'] = y

In [ ]:
brest_type1 = train_df_copy[train_df_copy['label_num']==5]

In [ ]:
brest_type3 = train_df_copy[train_df_copy['label_num']==0]
brest_type3.head()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_predict

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_feature_XGBC, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
train_df_copy[['ROPN1','label_num']]

In [ ]:
sns.scatterplot(
    data = train_df_copy[['ROPN1','label_num']],
    y= 'ROPN1',
    x = 'label_num',
    hue = 'label_num'
)

In [ ]:
df_ROPN1 = train_df_copy[['ROPN1','label_num']]
df_ROPN1 = df_ROPN1[(df_ROPN1['label_num'] == 0) | (df_ROPN1['label_num'] == 5)]

In [ ]:
df_ROPN1

In [ ]:
sns.barplot(
    data = df_ROPN1,
    x = 'label_num',
    y = 'ROPN1',
    hue = 'label_num'
)

In [ ]:
sns.scatterplot(
    data = train_df_copy[['ROPN1','label_num']],
    y= 'ROPN1',
    x = 'label_num',
    hue = 'label_num'
)

In [ ]:
df_ROPN1_breast_type3 = df_ROPN1[df_ROPN1['label_num'] == 0]
print(df_ROPN1_breast_type3)
df_ROPN1_breast_type3['ROPN1'].mean()

In [ ]:
df_ROPN1_breast_type1 = df_ROPN1[df_ROPN1['label_num'] == 5]
print(df_ROPN1_breast_type1)
df_ROPN1_breast_type1['ROPN1'].mean()

Удаляем ROPN - он будет мешать диффуренцировать

In [ ]:
df_VGLL1_full = train_df_copy[['VGLL1','label_num']]
df_VGLL1_full.sort_values (by = ['VGLL1'], ascending = [False])

In [ ]:
sns.barplot(
    data = df_VGLL1_full,
    x='label_num',
    y='VGLL1'
)

In [ ]:
df_VGLL1_brest = df_VGLL1[(df_VGLL1['label_num'] == 0) | (df_VGLL1['label_num'] == 5)]
df_VGLL1_brest

In [ ]:
sns.barplot(
    data = df_VGLL1_brest,
    x = 'label_num',
    y='VGLL1'
)

In [ ]:
df_VGLL1_brest[df_VGLL1_brest['label_num'] ==5].mean()

In [ ]:
df_brest5_Analys = train_df_copy[['FOXC1','SFRP1','LRRFIP2','ROPN1','OR4N2','RPL10L','label_num']]
df_brest5_Analys

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='FOXC1'
)

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='SFRP1'
)

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='SFRP1'
)

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='ROPN1'
)

https://www.proteinatlas.org/ENSG00000065371-ROPN1/pathology

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='ROPN1'
)

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='ROPN1'
)

In [ ]:
sns.barplot(
    data = df_brest5_Analys,
    x='label_num',
    y='RPL10L'
)

Думаю стоит поиграться с признаками (основное в пункте готовый датасет с признаками)

In [ ]:
#X_train_feature_XGBC['brest_type1'] = np.where(X_train_feature_XGBC['VGLL1'] > 29.027407, 150 , 1)

In [ ]:
#X_train_feature_XGBC['brest_type3'] = np.where(X_train_feature_XGBC['VGLL1'] < 29.027407, 1 , 0)

In [ ]:
#X_train_feature_XGBC = X_train_feature_XGBC.drop(columns = 'VGLL1')

*Данные действия добавили лишь небольшой процент точности, все-таки нужно поискать закономерности в признаках, чтобы отличить 1 и 5 предсказательный класс*****

**Поэтому попробуем грубый метод через логистическую регрессию**

In [ ]:
brest_type1_3 =train_df[(train_df['label'] == 'brest_type1') | (train_df['label'] == 'brest_type3')]

In [ ]:
brest_type3 = train_df[train_df['label'] == 'brest_type3']

In [ ]:
brest_type1_3

In [ ]:
from sklearn import linear_model

In [ ]:
log_reg_full = linear_model.LogisticRegression(random_state=42, max_iter=1000)

In [ ]:
X = brest_type1_3.drop('label', axis=1)
y = brest_type1_3['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
log_reg_full.fit(X_train, y_train)

In [ ]:
y_pred2 = log_reg_full.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred2))

In [ ]:
log_reg_full.feature_names_in_

In [ ]:
feature_lg = pd.Series(log_reg_full.coef_[0],index=log_reg_full.feature_names_in_).sort_values(ascending=False)
feature_lg = pd.DataFrame(data = feature_lg)

In [ ]:
feature_lg

In [ ]:
feature_lg[feature_lg[0]>0.020]

https://www.proteinatlas.org/humanproteome/tissue/breast#relevant_links_and_publications   - они частично соотносятся 

И попробуем различить на основе важности признаков от RFC

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Hyperparameter grid
params = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
forest = RandomForestClassifier(random_state = 42)

rs_brest = GridSearchCV(estimator=forest, param_grid=params, scoring='recall', cv=3)

# Fit
rs_brest.fit(X_train, y_train)

In [ ]:
import joblib
joblib.dump(rs_brest, 'rs_brest.pkl')

In [ ]:
import joblib
#rs_brest = joblib.load('rs_brest.pkl')

In [ ]:
rs_brest.best_params_

In [ ]:
best_model = rs_brest.best_estimator_

In [ ]:
best_params_RFC = rs_brest.best_params_

In [ ]:
RFC_brest = RandomForestClassifier(**best_params_RFC)
RFC_brest

In [ ]:
RFC_brest.fit(X_train, y_train)

In [ ]:
feature_imp = pd.Series(RFC_brest.feature_importances_,index=RFC_brest.feature_names_in_).sort_values(ascending=False)
feature_imp

In [ ]:
feature_df_brest = pd.DataFrame(data = feature_imp, columns =['score'])
feature_df_brest = feature_df_brest[feature_df_brest['score']>0.002]

In [ ]:
feature_df_brest.reset_index()

In [ ]:
feature_df_brest_final = list(feature_df_brest.index)
                              

In [ ]:
X_final_brest = train_df[feature_df_brest_final]

In [ ]:
X_final_brest['label'] = train_df['label']

In [ ]:
X_final_brest.columns

In [ ]:
import joblib
joblib.dump(rs_brest, 'rs_brest.pkl')

In [ ]:
import joblib
#rs_brest = joblib.load('rs_brest.pkl')

Но опять же я по времени не смог найти значимые фичи для отличия, чтобы дать модели большей предсказательности.